In [6]:
import rasterio
import pandas as pd
import numpy as np
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import RdYlGn,jet,RdBu

# Import Planetary Computer tools
import stackstac
import pystac_client
from planetary_computer import sign
from odc.stac import stac_load

ModuleNotFoundError: No module named 'rioxarray'

Data Extraction

### S2 
B01 = Coastal Aerosol = 60m <br>
B02 = Blue = 10m <br>
B03 = Green = 10m <br>
B04 = Red = 10m <br>
B05 = Red Edge (704 nm) = 20m <br>
B06 = Red Edge (740 nm) = 20m <br>
B07 = Red Edge (780 nm) = 20m <br>
B08 = NIR (833 nm) = 10m <br>
B8A = NIR (narrow 864 nm) = 20m <br>
B11 = SWIR (1.6 um) = 20m <br>
B12 = SWIR (2.2 um) = 20m


### Landsat Band Summary 
The following list of bands will be loaded by the Open Data Cube (ODC) stac command:<br>
We will use two load commands to separate the RGB data from the Surface Temperature data.<br><br>
Band 2 = blue = 30m<br>
Band 3 = green = 30m<br>
Band 4 = red = 30m<br>
Band 5 = nir08 (near infrared) = 30m<br>
Band 11 = Surface Temperature = lwir11 = 100m

In [265]:
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])
time_window = "2021-06-01/2021-09-01"

In [ ]:
#S2
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds, 
    datetime=time_window,
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 30}},
)

items = list(search.get_items())
print('This is the number of scenes that touch our region:',len(items))

resolution = 10  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 

s2 = stac_load(
    items,
    bands=["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=sign,
   bbox=bounds
)

In [101]:
data_slice = s2.isel(time=7)
height = data_slice.dims["latitude"]
width = data_slice.dims["longitude"]
gt = rasterio.transform.from_bounds(lower_left[1],lower_left[0],upper_right[1],upper_right[0],width,height)
data_slice.rio.write_crs("epsg:4326", inplace=True)
data_slice.rio.write_transform(transform=gt, inplace=True);

with rasterio.open('S2_bianca.tiff','w',driver='GTiff',width=width,height=height,
                   crs='epsg:4326',transform=gt,count=6,compress='lzw',dtype='float64') as dst:
    dst.write(data_slice.B02,1)
    dst.write(data_slice.B03,2)
    dst.write(data_slice.B04,3) 
    dst.write(data_slice.B08,4)
    dst.write(data_slice.B11,5) 
    dst.write(data_slice.B12,6)
    dst.close()

In [ ]:
#Lst
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bounds, 
    datetime=time_window,
    collections=["landsat-c2-l2"],
    query={"eo:cloud_cover": {"lt": 50},"platform": {"in": ["landsat-8"]}},
)

items = list(search.get_items())
print('This is the number of scenes that touch our region:',len(items))

data1 = stac_load(
    items,
    bands=["red", "green", "blue", "nir08"],
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=sign,
    bbox=bounds
)

data2 = stac_load(
    items,
    bands=["lwir11"],
    crs="EPSG:4326", # Latitude-Longitude
    resolution=scale, # Degrees
    chunks={"x": 2048, "y": 2048},
    dtype="uint16",
    patch_url=sign,
    bbox=bounds
)

# Persist the data in memory for faster operations
data1 = data1.persist()
data2 = data2.persist()

# Scale Factors for the RGB and NIR bands 
scale1 = 0.0000275 
offset1 = -0.2 
data1 = data1.astype(float) * scale1 + offset1

# Scale Factors for the Surface Temperature band
scale2 = 0.00341802 
offset2 = 149.0 
kelvin_celsius = 273.15 # convert from Kelvin to Celsius
data2 = data2.astype(float) * scale2 + offset2 - kelvin_celsius

# Only select one of the time slices to output
data3 = data2.isel(time=2)

In [103]:
# Calculate the dimensions of the file
height = data3.dims["latitude"]
width = data3.dims["longitude"]

gt = rasterio.transform.from_bounds(lower_left[1],lower_left[0],upper_right[1],upper_right[0],width,height)
data3.rio.write_crs("epsg:4326", inplace=True)
data3.rio.write_transform(transform=gt, inplace=True);

with rasterio.open("LST_bianca.tiff",'w',driver='GTiff',width=width,height=height,
                   crs='epsg:4326',transform=gt,count=1,compress='lzw',dtype='float64') as dst:
    dst.write(data3.lwir11,1)
    dst.close()

 Data Pre-processing

In [104]:
def calculate_indices_sentinel2(image_path):
    """
    Calculate comprehensive set of indices from Sentinel-2 imagery
    """
    with rasterio.open(image_path) as src:
        # Read bands (10m resolution)
        blue = src.read(1).astype(float)    # B02
        green = src.read(2).astype(float)   # B03
        red = src.read(3).astype(float)     # B04
        nir = src.read(4).astype(float)     # B08
        
        # Read 20m bands and resample
        swir1 = src.read(5).astype(float)  # B11
        swir2 = src.read(6).astype(float)  # B12

        
        # Calculate indices
        # 1. Vegetation indices
        ndvi = (nir - red) / (nir + red)
        evi = 2.5 * ((nir - red) / (nir + 6 * red - 7.5 * blue + 1))
        savi = ((nir - red) / (nir + red + 0.5)) * (1.5)  # Soil Adjusted VI
        
        # 2. Built-up area indices
        ndbi = (swir1 - nir) / (swir1 + nir)
        bui = ndbi - ndvi  # Built-Up Index
        
        # 3. Moisture indices
        ndwi = (green - nir) / (green + nir)
        mndwi = (green - swir1) / (green + swir1)
        
        # 4. Additional indices
        ui = (swir2 - nir) / (swir2 + nir)  # Urban Index
        ebi = (swir1 - nir) / (10 * np.sqrt(swir1 + nir))  # Enhanced Built-up Index
        
        return {
            'ndvi': ndvi,
            'evi': evi,
            'savi': savi,
            'ndbi': ndbi,
            'bui': bui,
            'ndwi': ndwi,
            'mndwi': mndwi,
            'ui': ui,
            'ebi': ebi
        }

def calculate_indices_landsat8(image_path):
    """
    Calculate indices from Landsat 8 imagery (without thermal band)
    """
    with rasterio.open(image_path) as src:
        #print(src.count)
        lwir = src.read(1).astype(float)
        return {
            'lwir': lwir
        }

In [ ]:
s2_idx = calculate_indices_sentinel2("S2_bianca.tiff")
lst_idx = calculate_indices_landsat8("LST_bianca.tiff")
uhi_data = pd.DataFrame(pd.read_csv('Training_data_uhi_index_UHI2025-v2.csv'))

print("X",len(s2_idx),"+", len(lst_idx),"y",len(uhi_data['UHI Index']))

In [ ]:
#len(s2_idx.values())
len(lst_idx.values())

Features Engineering

In [285]:
def prepare_training_data(s2_indices, l8_indices, uhi_df):
    """
    Prepare training data combining both satellites' indices
    """
    # Create feature matrix
    features = []
    
    # Add Sentinel-2 features
    for index in s2_indices.values():
        features.append(index.flatten())
        
    # Add Landsat 8 features
    for index in l8_indices.values():
        features.append(index.flatten())
    
    
    X = np.column_stack(features)
   
    # Matching uhi values
    uhi_values = []
   
    # Function to convert lat/lon to raster pixel coords
    with rasterio.open("S2_bianca.tiff") as src:
       transform = src.transform
       data_array = src.read(1) # Read LST data

    def get_pixel_coords(lon,  lat, transform):
        c,r = ~transform * (lon, lat)
        return int(r), int(c)
    
    for _, row in uhi_df.iterrows():
        try:
            r, c = get_pixel_coords(row['Longitude'],row['Latitude'],transform)
            
            if 0 <= r < data_array.shape[0] and 0 <= c < data_array.shape[1]:
                uhi_values.append(row['UHI Index'])
            else:
                continue # skip Out of bounds
            
        except IndexError:
            continue # skip outside outer bounds
            
    y = np.array(uhi_values)
    
    # Ensure x,y matching
    m_len = min(X.shape[0], len(y))
    X, y = X[:m_len], y[:m_len]
    
    return X, y,  list(s2_indices.keys()) + list(l8_indices.keys())

In [286]:
X, y, feature_names = prepare_training_data(s2_idx, lst_idx, uhi_data)

In [ ]:
# Save train dataset as train_data
train_df = pd.DataFrame(X, columns = feature_names)
train_df['UHI Index'] = y
train_df

In [289]:
train_df.to_pickle("train_dataset_v1.pkl")